<a href="https://colab.research.google.com/github/khadidja2005/Pytorch_practice/blob/main/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import os
import torchmetrics
from torchmetrics.classification import Accuracy

ModuleNotFoundError: No module named 'torchmetrics'